In [28]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
import requests
import time
import re

db = MongoClient().rsmarket

itemPricePages = range(1,68)
itemIndexPages = range(248,266)

In [29]:
def extractItemUrl(soupObj):
    source = soupObj.findAll('a',href=True)
    itemLinks = []
    for link in source:
        if re.match('http://www\.grandexchangewatch\.com/item/[0-9]+-',link['href']):
            itemLinks.append(link['href'])
    return itemLinks
def extractItemPrices(soupObj):
    source = BeautifulSoup(str(soupObj.find('table',{'id':"data-table"})))
    source = BeautifulSoup(str(source.findAll('tr',{"class":re.compile('tr-')})))
    source = source.findAll("td")

    tempList = []
    for i in source:
        tempList.append(i.contents[0])

    realList = []
    for i in range(0,len(tempList),3):
        realList.append((tempList[i],tempList[i+1],tempList[i+2]))
    return realList
def extractItemName(soupObj):
    source = soup.findAll("h1")
    return source[0].contents[0]['title']

def extractItemInfo(soupObj):
    source = BeautifulSoup(str(soupObj.findAll('div',{'class':'float-right'})))
    source = BeautifulSoup(str(source.findAll('table',{'id':"data-table"})))
    source = BeautifulSoup(str(source.findAll('tr',{"class":re.compile('tr-')})))
    source = source.findAll("td")
    vals = []
    for i in source[12:24][1::2]:
        vals.append(i.contents[0])
    return vals

#extractItemPrices(BeautifulSoup(requests.get(extractItemUrl(soup)[3]).content))

In [30]:
for indexPage in itemIndexPages:
    page = requests.get("http://www.grandexchangewatch.com/item-db/?query=&sort=popular&dir=desc&show=both&min=&max=&start="+str(indexPage)+"#.VYozi5VVikp").content
    soup = BeautifulSoup(page)
    for itemUrl in extractItemUrl(soup)[::2]:#get item urls from index page
        document = {}
        page = requests.get(itemUrl).content
        soup = BeautifulSoup(page)
        document['ItemName']=extractItemName(soup)
        info = extractItemInfo(soup)
        document['MembersOnly']=info[0]
        document['DateAdded']=info[1]
        document['Category']=info[3]
        document['LowAlch']=info[4]
        document['HighAlch']=info[5]
        priceList = []
        time.sleep(0.5)
        for pricePage in itemPricePages:
            page = requests.get(str(itemUrl)+"?start="+str(pricePage)).content
            soup = BeautifulSoup(page)
            priceList = priceList+extractItemPrices(soup)
            time.sleep(0.5)
        document['Prices']=priceList
        db.items.insert(document)
        print indexPage

TypeError: string indices must be integers